In [4]:
import pandas as pd
import numpy as np
import sqlalchemy as sa              #Para conexión a BD, requerido para usar pd.read_sql()
from sqlalchemy import text
import urllib                        #Para formatear string de conexión
#import cargahistorica as ch          #Carga el módulo en que definí la rutina de actualización de los datos 

param_DW = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};SERVER=10.34.71.202;UID=datawarehouse;PWD=datawarehouse;DATABASE=Estudios;TrustServerCertificate=yes")
engine = sa.create_engine("mssql+pyodbc:///?odbc_connect=%s" % param_DW)


In [5]:
# Formatea los datos obtenidos desde Energía Abierta para el año 2024
# Este notebook llena la brecha de información 
# que quedó entre el 16 de abril y el 27 de mayo, que es el momento en que se automatizó la consulta a la API de la CNE 

data_2024 = pd.read_csv("C:/o/DCCP/Javier Guajardo - Traspaso/2. Cálculo de Ahorro/savings_dccp/data/combustible/precios por estacion/energia_abierta/2024.csv/2024.csv", sep=",")

#start_date = '2024-04-16'
#end_date = '2024-05-27'
#data_2024 = data_2024.loc[(data_2024['fecha_actualizacion'] >= start_date) & (data_2024['fecha_actualizacion'] <= end_date)]


C:\Users\hector.garrido\AppData\Local\Temp\ipykernel_5308\847562819.py:5: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  data_2024 = pd.read_csv("C:/o/DCCP/Javier Guajardo - Traspaso/2. Cálculo de Ahorro/savings_dccp/data/combustible/precios por estacion/energia_abierta/2024.csv/2024.csv", sep=",")


In [6]:
reemplazos = {
    'DI':'Diesel',
    '93':'Bencina93',
    '95':'Bencina95',
    '97':'Bencina97'
}

def modify_data(data,fueltype, reemplazos):
    renamed_data = data.rename(columns={'codigo':'id','fecha_actualizacion':'fecha_inicio','combustible':'tipo'})
    renamed_data['fecha_fin'] = pd.Timestamp.now().strftime('%Y-%m-%d')
    reordered_data = renamed_data[['id','fecha_inicio','fecha_fin','precio','tipo','hora_actualizacion']]
    reordered_data['tipo'] = reordered_data['tipo'].replace(reemplazos)
    modified_data = reordered_data[reordered_data['tipo'].isin(reemplazos.values())]
    filtered_data = modified_data[modified_data['tipo']==fueltype]
    return filtered_data

Bencina93_24, Bencina95_24, Bencina97_24, Diesel_24 = (modify_data(data_2024, name, reemplazos) for name in ['Bencina93', 'Bencina95', 'Bencina97', 'Diesel'])


C:\Users\hector.garrido\AppData\Local\Temp\ipykernel_5308\3406761952.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reordered_data['tipo'] = reordered_data['tipo'].replace(reemplazos)
C:\Users\hector.garrido\AppData\Local\Temp\ipykernel_5308\3406761952.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reordered_data['tipo'] = reordered_data['tipo'].replace(reemplazos)
C:\Users\hector.garrido\AppData\Local\Temp\ipykernel_5308\3406761952.py:12: SettingWithCopyWarning: 
A value is trying to be set on 

In [ ]:
# Paso 2: Cargar los nuevos datos en la tabla auxiliar
Bencina93_24[['id','precio','tipo','fecha_inicio','fecha_fin']].to_sql('Bencina93Hist', con=engine, if_exists='append', index=False, dtype={
        'id': sa.types.NVARCHAR(length=50),
        'precio': sa.types.Float(),
        'tipo': sa.types.NVARCHAR(length=50),
        'fecha_inicio': sa.types.Date(),
        'fecha_fin': sa.types.Date()
})

In [12]:
# Paso 2: Cargar los nuevos datos en la tabla auxiliar
Bencina95_24[['id','precio','tipo','fecha_inicio','fecha_fin']].to_sql('Bencina95Hist', con=engine, if_exists='append', index=False, dtype={
        'id': sa.types.NVARCHAR(length=50),
        'precio': sa.types.Float(),
        'tipo': sa.types.NVARCHAR(length=50),
        'fecha_inicio': sa.types.Date(),
        'fecha_fin': sa.types.Date()
})

# Paso 2: Cargar los nuevos datos en la tabla auxiliar
Bencina97_24[['id','precio','tipo','fecha_inicio','fecha_fin']].to_sql('Bencina97Hist', con=engine, if_exists='append', index=False, dtype={
        'id': sa.types.NVARCHAR(length=50),
        'precio': sa.types.Float(),
        'tipo': sa.types.NVARCHAR(length=50),
        'fecha_inicio': sa.types.Date(),
        'fecha_fin': sa.types.Date()
})

# Paso 2: Cargar los nuevos datos en la tabla auxiliar
Diesel_24[['id','precio','tipo','fecha_inicio','fecha_fin']].to_sql('DieselHist', con=engine, if_exists='append', index=False, dtype={
        'id': sa.types.NVARCHAR(length=50),
        'precio': sa.types.Float(),
        'tipo': sa.types.NVARCHAR(length=50),
        'fecha_inicio': sa.types.Date(),
        'fecha_fin': sa.types.Date()
})

-1

In [8]:
# Paso 2: Cargar los nuevos datos en la tabla auxiliar
Bencina93_24.to_sql('Bencina93Paso', con=engine, if_exists='append', index=False, dtype={
        'id': sa.types.NVARCHAR(length=50),
        'precio': sa.types.Float(),
        'tipo': sa.types.NVARCHAR(length=50),
        'fecha_inicio': sa.types.Date(),
        'fecha_fin': sa.types.Date(),
        'hora_actualizacion':sa.types.Time()
})

Bencina95_24.to_sql('Bencina95Paso', con=engine, if_exists='append', index=False, dtype={
        'id': sa.types.NVARCHAR(length=50),
        'precio': sa.types.Float(),
        'tipo': sa.types.NVARCHAR(length=50),
        'fecha_inicio': sa.types.Date(),
        'fecha_fin': sa.types.Date(),
        'hora_actualizacion':sa.types.Time()
})

Bencina97_24.to_sql('Bencina97Paso', con=engine, if_exists='append', index=False, dtype={
        'id': sa.types.NVARCHAR(length=50),
        'precio': sa.types.Float(),
        'tipo': sa.types.NVARCHAR(length=50),
        'fecha_inicio': sa.types.Date(),
        'fecha_fin': sa.types.Date(),
        'hora_actualizacion':sa.types.Time()
})

Diesel_24.to_sql('DieselPaso', con=engine, if_exists='append', index=False, dtype={
        'id': sa.types.NVARCHAR(length=50),
        'precio': sa.types.Float(),
        'tipo': sa.types.NVARCHAR(length=50),
        'fecha_inicio': sa.types.Date(),
        'fecha_fin': sa.types.Date(),
        'hora_actualizacion':sa.types.Time()
})

-1